Description
===========

Docker container for running OCRopus commands. This bundles up:

    - CUDA and PyTorch
    - A Python 2.7 Conda installation
    - The major modules from OCRopus3

PyTorch is currently at version 0.3.1, since OCRopus hasn't been converted yet
to PyTorch 0.4.

You need to set a Jupyter password (see `jupyter_config/README`) and
a VNC password (see `vnc_config/README`).

To clone this repository, use the `--recursive` flag:

    git clone --recursive git@github.com:tmbdev/ocropus3-docker.git
    cd ocropus3-docker
    ./build

The `ocropus3-docker` repository builds a Docker container that you can use to run OCRopus3 on any platform.

- `./build` -- build the container
- `./ocropy` -- run the container

The container automatically starts a VNC server for graphical output. Inside the container is a complete OCRopus3 installation.

# Docker Container

Some notes on the Docker container:

- the pytorch version is 0.3.1; OCRopus hasn't been ported to 0.4 yet
- the Python installation is in /opt/conda, separate from the regular Ubuntu installation

In [2]:
!head Dockerfile; echo ...; tail Dockerfile

FROM nvidia/cuda:9.0-base
#FROM nvidia/cuda:9.1-base
#FROM nvidia/cuda:9.2-devel-ubuntu18.04
MAINTAINER Tom Breuel <tmbdev@gmail.com>

ENV DEBIAN_FRONTEND noninteractive
ENV DEBCONF_NONINTERACTIVE_SEEN true

RUN apt-get -y update

...
ADD scripts/* /usr/local/bin/

RUN true \
    && echo ". /opt/conda/etc/profile.d/conda.sh" >> $HOME/.bashrc \
    && echo "conda activate base" >> $HOME/.bashrc \
    && chown -R $USER.$USER $HOME
RUN echo 'user ALL=(ALL:ALL) NOPASSWD:ALL' >> /etc/sudoers

USER $UID
ENTRYPOINT ["runcmd"]


In [3]:
!./build





Sending build context to Docker daemon  703.2MB
Step 1/67 : FROM nvidia/cuda:9.0-base
 ---> f3a8582463d4
Step 2/67 : MAINTAINER Tom Breuel <tmbdev@gmail.com>
 ---> Using cache
 ---> 488754540ac4
Step 3/67 : ENV DEBIAN_FRONTEND noninteractive
 ---> Using cache
 ---> 01f341ae724c
Step 4/67 : ENV DEBCONF_NONINTERACTIVE_SEEN true
 ---> Using cache
 ---> 18e13a972ea2
Step 5/67 : RUN apt-get -y update
 ---> Using cache
 ---> 8adf79ab81c5
Step 6/67 : RUN  apt-get -y install sudo lsb-release build-essential curl software-properties-common     && echo "deb http://packages.cloud.google.com/apt cloud-sdk-$(lsb_release -c -s) main"            >> /etc/apt/sources.list.d/google-cloud-sdk.list     && curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key  add -     && apt-get update -y && apt-get dist-upgrade -y && apt-get -y install apt-utils     && apt-get -y install locales && locale-gen en_US.UTF-8 && dpkg-reconfigure locales
 ---> Using cache
 ---> 8c7bdef331e8
Step 7/67 :

Removing intermediate container 2cff04a900f3
 ---> 07adbe16f177
Step 17/67 : RUN conda install simplejson
 ---> Running in e825dcc82f91
Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs: 
    - simplejson


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    simplejson-3.15.0          |   py27h14c3975_0         101 KB

The following NEW packages will be INSTALLED:

    simplejson: 3.15.0-py27h14c3975_0

Proceed ([y]/n)? 
simplejson-3.15.0    |  101 KB | ########## | 100% 

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Removing intermediate container e825dcc82f91
 ---> d3e6df3cad46
Step 18/67 : RUN conda install pyzmq
 ---> Running in fcaf91cf6f72
Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

 

dbus-1.13.2          |  554 KB | ########## | 100% 
webencodings-0.5.1   |   19 KB | ########## | 100% 
terminado-0.8.1      |   20 KB | ########## | 100% 
backports-1.0        |    3 KB | ########## | 100% 
ipykernel-4.8.2      |  143 KB | ########## | 100% 
nbformat-4.4.0       |  136 KB | ########## | 100% 
ipython-5.7.0        |  1.0 MB | ########## | 100% 
xz-5.2.4             |  366 KB | ########## | 100% 
sip-4.19.8           |  291 KB | ########## | 100% 
backports_abc-0.5    |   12 KB | ########## | 100% 
freetype-2.8         |  804 KB | ########## | 100% 
qtconsole-4.3.1      |  147 KB | ########## | 100% 
jpeg-9b              |  248 KB | ########## | 100% 
markupsafe-1.0       |   24 KB | ########## | 100% 
backports.shutil_get |    8 KB | ########## | 100% 
gst-plugins-base-1.1 |  6.3 MB | ########## | 100% 
python-dateutil-2.7. |  258 KB | ########## | 100% 
pcre-8.42            |  251 KB | ########## | 100% 
testpath-0.3.1       |   89 KB | ########## | 100% 
pexpect-4.6.

dask-0.17.5          |    3 KB | ########## | 100% 
pyyaml-3.12          |  159 KB | ########## | 100% 
distributed-1.21.8   |  748 KB | ########## | 100% 
pyparsing-2.2.0      |   93 KB | ########## | 100% 
bokeh-0.12.16        |  4.2 MB | ########## | 100% 
tblib-1.3.2          |   15 KB | ########## | 100% 
toolz-0.9.0          |   90 KB | ########## | 100% 
packaging-17.1       |   32 KB | ########## | 100% 
matplotlib-2.2.2     |  6.5 MB | ########## | 100% 
olefile-0.45.1       |   47 KB | ########## | 100% 
scikit-image-0.13.1  | 23.2 MB | ########## | 100% 
pandas-0.23.0        | 11.8 MB | ########## | 100% 
psutil-5.4.5         |  297 KB | ########## | 100% 
locket-0.2.0         |    8 KB | ########## | 100% 
heapdict-1.0.0       |    8 KB | ########## | 100% 
sortedcontainers-2.0 |   41 KB | ########## | 100% 
zict-0.1.3           |   18 KB | ########## | 100% 
cytoolz-0.9.0.1      |  410 KB | ########## | 100% 
backports.functools_ |    9 KB | ########## | 100% 
pytz-2018.4 

pytorch-0.3.1        | 486.9 MB | ########## | 100% 

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Removing intermediate container d29415dc80e9
 ---> 02bc7bf3f928
Step 25/67 : RUN conda install msgpack
 ---> Running in ce3c9931064c
Solving environment: ...working... done

# All requested packages already installed.

Removing intermediate container ce3c9931064c
 ---> 155c118ce605
Step 26/67 : RUN conda install -c conda-forge google-cloud-storage
 ---> Running in 8db4786073a6
Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs: 
    - google-cloud-storage


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2o             |                0         3.5 MB  conda-forge
    google-resumable-media-0.3.1|             py_0          28 KB 

openssl-1.0.2o       |  3.5 MB | ########## | 100% 
google-resumable-med |   28 KB | ########## | 100% 
google-cloud-core-0. |   23 KB | ########## | 100% 
pyasn1-0.4.3         |   46 KB | ########## | 100% 
certifi-2018.4.16    |  142 KB | ########## | 100% 
googleapis-common-pr |   51 KB | ########## | 100% 
pyasn1-modules-0.2.1 |   33 KB | ########## | 100% 
google-auth-1.5.0    |   39 KB | ########## | 100% 
protobuf-3.5.2       |  1.3 MB | ########## | 100% 
ca-certificates-2018 |  139 KB | ########## | 100% 
libprotobuf-3.5.2    |  4.5 MB | ########## | 100% 
google-api-core-0.1. |   36 KB | ########## | 100% 
google-cloud-storage |   40 KB | ########## | 100% 
cachetools-2.0.1     |   10 KB | ########## | 100% 
conda-4.5.4          |  629 KB | ########## | 100% 
rsa-3.4.2            |   50 KB | ########## | 100% 

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Removing intermediate container 8db478607

Removing intermediate container 9035b52b4bca
 ---> e4e8f4fb301d
Step 34/67 : COPY dltrainers /tmp/dltrainers
 ---> 5bc80d54ad87
Step 35/67 : RUN cd /tmp/dltrainers && python setup.py install && rm -rf /tmp/dltrainers
 ---> Running in a7acdafef997
running install
running build
running build_py
creating build
creating build/lib
creating build/lib/dltrainers
copying dltrainers/layers.py -> build/lib/dltrainers
copying dltrainers/helpers.py -> build/lib/dltrainers
copying dltrainers/__init__.py -> build/lib/dltrainers
copying dltrainers/flex.py -> build/lib/dltrainers
copying dltrainers/trainers.py -> build/lib/dltrainers
running install_lib
creating /opt/conda/lib/python2.7/site-packages/dltrainers
copying build/lib/dltrainers/layers.py -> /opt/conda/lib/python2.7/site-packages/dltrainers
copying build/lib/dltrainers/helpers.py -> /opt/conda/lib/python2.7/site-packages/dltrainers
copying build/lib/dltrainers/__init__.py -> /opt/conda/lib/python2.7/site-packages/dltrainers
copying build/li

Removing intermediate container 4392eef7778d
 ---> d0c9c8312f63
Step 42/67 : COPY ocrodeg /tmp/ocrodeg
 ---> 212d9c32c4d0
Step 43/67 : RUN cd /tmp/ocrodeg && python setup.py install && rm -rf /tmp/ocrodeg
 ---> Running in f83a59dd8a2f
running install
running build
running build_py
creating build
creating build/lib
creating build/lib/ocrodeg
copying ocrodeg/degrade.py -> build/lib/ocrodeg
copying ocrodeg/__init__.py -> build/lib/ocrodeg
running install_lib
creating /opt/conda/lib/python2.7/site-packages/ocrodeg
copying build/lib/ocrodeg/degrade.py -> /opt/conda/lib/python2.7/site-packages/ocrodeg
copying build/lib/ocrodeg/__init__.py -> /opt/conda/lib/python2.7/site-packages/ocrodeg
byte-compiling /opt/conda/lib/python2.7/site-packages/ocrodeg/degrade.py to degrade.pyc
byte-compiling /opt/conda/lib/python2.7/site-packages/ocrodeg/__init__.py to __init__.pyc
running install_egg_info
Writing /opt/conda/lib/python2.7/site-packages/ocrodeg-v0.0-py2.7.egg-info
Removing intermediate container

 ---> Running in b999151d829f
Removing intermediate container b999151d829f
 ---> 3ea5ec44c36d
Step 56/67 : ENV NVIDIA_DRIVER_CAPABILITIES compute,utility
 ---> Running in fb9768ccb380
Removing intermediate container fb9768ccb380
 ---> 6d5c5632ee2b
Step 57/67 : COPY nginx.conf /etc/nginx
 ---> 2bd28ae908bb
Step 58/67 : EXPOSE 3218
 ---> Running in d279ec43847f
Removing intermediate container d279ec43847f
 ---> c5c7e835c19d
Step 59/67 : RUN mkdir -p $HOME && groupadd -g $GID -r $USER && useradd --no-log-init -u $UID -r -g $USER $USER
 ---> Running in d2d3916f6b3b
Removing intermediate container d2d3916f6b3b
 ---> 5ee20ccdaab9
Step 60/67 : RUN mkdir -p $HOME/.jupyter $HOME/.vnc $HOME/.ssh
 ---> Running in 563522b6e55b
Removing intermediate container 563522b6e55b
 ---> 0a912ac7a04a
Step 61/67 : ADD jupyter_config/* $HOME/.jupyter/
 ---> 8d4200a165cc
Step 62/67 : ADD vnc_config/* $HOME/.vnc/
 ---> 6f1fd55c6a7e
Step 63/67 : ADD scripts/* /usr/local/bin/
 ---> db0c8137466e
Step 64/67 : RUN tr

Segmentation
============

Train a model with:

    ./ocropy ocroseg-train -d http://storage.googleapis.com/lpr-ocr/uw3-framed-lines.tgz

Models will be saved in the current directory.

You can view the training progress by connecting using VNC:

    xtightvncviewer :99

Models are saved in the current directory by default.

In [1]:
!./ocropy ocroseg-train -d http://storage.googleapis.com/lpr-ocr/uw3-framed-lines.tgz

+ ocroseg-train -d http://storage.googleapis.com/lpr-ocr/uw3-framed-lines.tgz
raw sample:
__key__ 'A001BIN'
__source__ 'http://storage.googleapis.com/lpr-ocr/uw3-framed-lines.tgz'
lines.png float32 (3300, 2592)
png float32 (3300, 2592)

preprocessed sample:
__key__ <type 'list'> ['A005BIN']
__source__ <type 'list'> ['http://storage.googleapis.com/lpr-ocr/uw3-framed-lines.tgz
input float32 (1, 3300, 2592, 1)
mask float32 (1, 3300, 2592, 1)
output float32 (1, 3300, 2592, 1)

ntrain 0
model:
Sequential(
  (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (4): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True)
  (6): ReLU()
  (7): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (8): Conv2d(20, 40, kerne

Line Recognition
================

Train a model with:

    ./ocropy ocroline-train -d http://storage.googleapis.com/lpr-ocr/uw3-dew-training.tgz \
            -t http://storage.googleapis.com/lpr-ocr/uw3-dew-testing.tgz

Models will be saved in the current directory.

You can view the training progress by connecting using VNC:

    xtightvncviewer :99

Models are saved in the current directory by default.

# Kubernetes

To run `ocropus3` on Kubernetes, you need to do the following:

- log into Google, set up Config.sh according to your project
- start up a Kubernetes cluster (`ku init`)
- submit your training job(s) (`kubectl apply -f ocroline-train.yaml`)

On GKE (Google Kubernetes Engine), you...

- write a job description in a YAML file
- use gs:// or http://storage.googleapis.com for your input shards
- save your models periodically to a Google storage bucket

In [1]:
!cat Config.sh

cluster=tmblearn
zone=us-central1-f
project=research-191823
image=gcr.io/$project/ocropy
cpu_machine=n1-standard-8
cpu_nodes=3
gpu_machine=n1-standard-16
gpu_nodes=2


In [3]:
!ku help

init -- initialize the cluster
daemonset -- start the NVIDIA daemonset
status -- cluster status
pods -- node list
stats -- node stats
build -- build the cloud image
kill -- kill the cluster
connect -- connect to a cluster
forward -- connect to a cluster
help -- display this help


In [4]:
!ku status

NAME      LOCATION       MASTER_VERSION  MASTER_IP       MACHINE_TYPE   NODE_VERSION  NUM_NODES  STATUS
tmblearn  us-central1-f  1.9.7-gke.1     104.198.252.27  n1-standard-8  1.9.7-gke.1   5          RUNNING
NAME          MACHINE_TYPE    DISK_SIZE_GB  NODE_VERSION
default-pool  n1-standard-8   100           1.9.7-gke.1
p100          n1-standard-16  100           1.9.7-gke.1


In [5]:
!ku stats

NAME      LOCATION       MASTER_VERSION  MASTER_IP       MACHINE_TYPE   NODE_VERSION  NUM_NODES  STATUS
tmblearn  us-central1-f  1.9.7-gke.1     104.198.252.27  n1-standard-8  1.9.7-gke.1   5          RUNNING

NAME         TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.27.240.1   <none>        443/TCP   1d

      1 ocroline-train Running
      1 ocroseg-train Running


In [8]:
!cat ocroseg-train.yaml

apiVersion: batch/v1
kind: Job
metadata:
  name: ocroseg-train
spec:
  template:
    spec:
      containers:
      - name: ocroseg-train
        image: gcr.io/research-191823/ocropy
        workingDir: "/tmp"
        command: ["/usr/local/bin/runcmd"]
        args:
        - ocroseg-train
        - "-d"
        - "http://storage.googleapis.com/lpr-ocr/uw3-framed-lines.tgz"
        - "-o"
        - "ocroseg"
        resources:
          requests:
          limits:
            nvidia.com/gpu: "1"
            cpu: 12
            memory: "48000Mi"
      nodeSelector:
        cloud.google.com/gke-accelerator: nvidia-tesla-p100
      restartPolicy: Never
  backoffLimit: 4


In [9]:
!gsutil ls gs://lpr-ocr/ | grep uw3

gs://lpr-ocr/_uw3-patches.tgz
gs://lpr-ocr/uw3-dew-testing.tgz
gs://lpr-ocr/uw3-dew-training.tgz
gs://lpr-ocr/uw3-framed-lines-test.tgz
gs://lpr-ocr/uw3-framed-lines-train.tgz
gs://lpr-ocr/uw3-framed-lines.tgz
gs://lpr-ocr/uw3-framed-zones.tgz
gs://lpr-ocr/uw3-lines-dew.tgz
gs://lpr-ocr/uw3-lines.tgz
gs://lpr-ocr/uw3-pages-test.tgz
gs://lpr-ocr/uw3-pages-train.tgz
gs://lpr-ocr/uw3-zones.tgz
gs://lpr-ocr/_uw3-patches/
gs://lpr-ocr/uw3-lines-old/


In [1]:
!ku connect ocroline ls | head

connecting to: ocroline-train-7fshb
Miniconda2-latest-Linux-x86_64.sh  ol-000000665-010886.pt
ol-000000005-239618.pt		   ol-000000670-007518.pt
ol-000000010-182026.pt		   ol-000000675-007790.pt
ol-000000015-119865.pt		   ol-000000680-008311.pt
ol-000000020-094170.pt		   ol-000000685-007108.pt
ol-000000025-079694.pt		   ol-000000690-010020.pt
ol-000000030-048406.pt		   ol-000000695-006404.pt
ol-000000035-058577.pt		   ol-000000700-007747.pt
ol-000000040-039189.pt		   ol-000000705-007729.pt
